In [1]:
! pip install transformers datasets
from transformers import AutoTokenizer
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling
from transformers import TFAutoModelForMaskedLM
from transformers import create_optimizer
import tensorflow as tf
import math

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:

imdb_dataset = load_dataset("imdb")
imdb_dataset

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [3]:
imdb_dataset['train'].shuffle(7)[0:4]

{'text': ['This interesting feature has a very fine story-line, rather colorful characters and a very steady pace. it also incorporates a plot device from "Reap the Wild Wind", and since Cecil B. Deille directed that and his son-in-law Anbthony Quinn directed this film from his preparations, that can hardly be a coincidence. it works in both cases, I must report. The unusual set-up tells the viewer that Barataria, an island ruled by Jean Lafitte is built upon piracy, but during the war of 1812, and before, he has always refrained from bothering United States\' vessels. Now General Andrew Jackson has been charged with defending nearby New Orleans with only 12,000 men against 60,000 British Imperial redcoats and 60 ships. Lafitte\'s men want him to side with the stronger force; he wants freedom and pardons for his men before ceding such a strategic landing spot to the U.S. forces. There are other factors at work in the story-line; pirate Bonnie Brown and her father want to attack U.S. sh

In [4]:
imdb_dataset=imdb_dataset['train']
imdb_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [5]:
model_checkpoint = "distilbert-base-uncased"

tokenizer=AutoTokenizer.from_pretrained(model_checkpoint)

In [6]:
def tokeizer_func(example):
    return tokenizer(example['text'])
imdb_dataset=imdb_dataset.map(tokeizer_func,batched=True,remove_columns=['text','label'])

  0%|          | 0/25 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors


In [7]:
cunck_size=128
def chuncking(example):
    result={k:sum(example[k],[]) for k in example.keys()}
    total_len = len(result[list(example.keys())[0]])

    total_len=(total_len//cunck_size)*cunck_size
    chunck={k:[v[i:i+cunck_size]for i in range(0,total_len,cunck_size)]for k,v in result.items()}
    chunck['labels']=chunck['input_ids'].copy()
    return chunck
imdb_dataset=imdb_dataset.map(chuncking,batched=True)

  0%|          | 0/25 [00:00<?, ?ba/s]

In [8]:
imdb_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 61291
})

In [9]:
data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm_probability=0.15,return_tensors='tf')

In [10]:
imdb_dataset=imdb_dataset.train_test_split(train_size=10000,test_size=2000,seed=42)
intermediate=imdb_dataset['test'].train_test_split(train_size=1000,test_size=1000,seed=42)
imdb_dataset['test']=intermediate['train']
imdb_dataset['validation']=intermediate['test']
imdb_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})

In [11]:
print(len(imdb_dataset['train']['labels'][-1]))
print(len(imdb_dataset['test']['labels'][-1]))
print(len(imdb_dataset['validation']['labels'][-1]))

128
128
128


In [12]:
samples = [imdb_dataset["train"][i] for i in range(2)]
batch = data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



'>>> , so that all you [MASK] for an hour and a half is a high [MASK] pitched whine going on [MASK] how his politics are best, or what [MASK]rky [MASK] of popular culture is tickling his fancy at the moment. he also finds airfields to [MASK]ag [MASK] films that he doesn't like [MASK] something [MASK] thought was reserved [MASK] losers murals me. surely being in a position like his [MASK]'d [MASK] he'd try and make a point about cinema a little more intelligently than this. by making a proper [MASK] perhaps, one with some ideas and a decent structure [MASK] [MASK] maybe one that isn't completely dominated by his annoying'

'>>> started [MASK] this was worthy of a 7, but as the film went on it dropped rapidly to a [MASK], then earning [MASK] 3 after the [MASK]lliness of the wedding scene. this was about as cold and sterile a movie as i have seen. a terrible waste of a good story. [SEP] [CLS] one thing that astonished me about this film [MASK] [MASK] not in a good way [MASK] was [MASK] n

In [13]:
train_data=imdb_dataset['train'].to_tf_dataset(columns=['input_ids', 'attention_mask'],
    batch_size=32,
    shuffle=True,
    collate_fn=data_collator,
    )
validation_data=imdb_dataset['validation'].to_tf_dataset(columns=['input_ids', 'attention_mask'],
    batch_size=32,
    shuffle=True,
    collate_fn=data_collator,
    )
test_data=imdb_dataset['test'].to_tf_dataset(columns=['input_ids', 'attention_mask'],
    batch_size=32,
    shuffle=True,
    collate_fn=data_collator,
    )

In [23]:
model=TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)
model.summary()

All PyTorch model weights were used when initializing TFDistilBertForMaskedLM.

All the weights of TFDistilBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.


Model: "tf_distil_bert_for_masked_lm_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 vocab_transform (Dense)     multiple                  590592    
                                                                 
 vocab_layer_norm (LayerNorm  multiple                 1536      
 alization)                                                      
                                                                 
 vocab_projector (TFDistilBe  multiple                 23866170  
 rtLMHead)                                                       
                                                                 
Total params: 66,985,530
Trainable params: 66,985,530
Non-trainable params: 0
________________________

In [22]:
from transformers import pipeline
model=pipeline('fill-mask',tokenizer=tokenizer,model=model)
text='This is a great [MASK].'
results=model(text)
for result in results:
    print(result)



{'score': 0.036511972546577454, 'token': 3066, 'token_str': 'deal', 'sequence': 'this is a great deal.'}
{'score': 0.02395866997539997, 'token': 3112, 'token_str': 'success', 'sequence': 'this is a great success.'}
{'score': 0.023744679987430573, 'token': 6172, 'token_str': 'adventure', 'sequence': 'this is a great adventure.'}
{'score': 0.016084974631667137, 'token': 2801, 'token_str': 'idea', 'sequence': 'this is a great idea.'}
{'score': 0.010877510532736778, 'token': 8658, 'token_str': 'feat', 'sequence': 'this is a great feat.'}


In [28]:
epoch=3

num_train_steps = len(train_data)
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)


In [25]:

eval_loss = model.evaluate(test_data)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

31/31 [==============================] - 8s 215ms/step - loss: 2.7327
Perplexity: 15.37


In [29]:
model.fit(train_data,validation_data=test_data,epochs=epoch)


Epoch 1/3
312/312 [==============================] - 200s 615ms/step - loss: 2.5258 - val_loss: 2.2226
Epoch 2/3
312/312 [==============================] - 194s 622ms/step - loss: 2.3376 - val_loss: 2.1596
Epoch 3/3
312/312 [==============================] - 190s 610ms/step - loss: 2.2659 - val_loss: 2.1415


In [30]:
eval_loss = model.evaluate(test_data)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

31/31 [==============================] - 7s 216ms/step - loss: 2.1606
Perplexity: 8.68


In [31]:
from transformers import pipeline
model=pipeline('fill-mask',tokenizer=tokenizer,model=model)

In [32]:
text='This is a great [MASK].'
results=model(text)
for result in results:
    print(result)



{'score': 0.11367155611515045, 'token': 2143, 'token_str': 'film', 'sequence': 'this is a great film.'}
{'score': 0.07091952115297318, 'token': 3185, 'token_str': 'movie', 'sequence': 'this is a great movie.'}
{'score': 0.0579768568277359, 'token': 2801, 'token_str': 'idea', 'sequence': 'this is a great idea.'}
{'score': 0.02580817975103855, 'token': 6172, 'token_str': 'adventure', 'sequence': 'this is a great adventure.'}
{'score': 0.024042371660470963, 'token': 3066, 'token_str': 'deal', 'sequence': 'this is a great deal.'}


In [ ]:
# you can see change in prediction and knowledge of model 